In [1]:
import xarray as xr
import numpy as np
import pandas as pd

import cftime
from datetime import date


In [2]:
filePath = '/glade/collections/cdg/data/cordex/data/raw/NAM-22i/day/RegCM4/HadGEM2-ES/hist/tasmax/tasmax.hist.HadGEM2-ES.RegCM4.day.NAM-22i.raw.nc'
ds = xr.open_dataset(filePath)
ds

<xarray.Dataset>
Dimensions:    (bnds: 2, lat: 258, lon: 600, time: 20160)
Coordinates:
  * time       (time) object 1950-01-01 12:00:00 ... 2005-12-30 12:00:00
  * lat        (lat) float64 12.12 12.38 12.62 12.88 ... 75.62 75.88 76.12 76.38
  * lon        (lon) float64 -171.9 -171.6 -171.4 ... -22.62 -22.38 -22.12
Dimensions without coordinates: bnds
Data variables:
    tasmax     (time, lat, lon) float32 ...
    time_bnds  (time, bnds) object ...
Attributes:
    CDI:                            Climate Data Interface version 1.7.2 (htt...
    Conventions:                    CF-1.4
    history:                        Thu Mar  5 17:39:59 2020: ncrcat -O -o /g...
    institution:                    Iowa State University
    contact:                        Raymond Arritt, rwarritt@bruce.agron.iast...
    creation_date:                  2017-01-01T00:00:00Z
    experiment:                     historical
    experiment_id:                  historical
    driving_experiment:             HadGEM2-ES,historical,r1i1p1
    driving_model_id:               HadGEM2-ES
    driving_model_ensemble_member:  r1i1p1
    driving_experiment_name:        historical
    frequency:                      day
    institute_id:                   ISU
    model_id:                       NCAR-RegCM4
    rcm_version_id:                 v4
    project_id:                     CORDEX
    CORDEX_domain:                  NAM-22
    references:                      
    product:                        output
    tracking_id:                    a7859e40-5d07-40aa-b675-1cdfbdb274e7
    NCO:                            netCDF Operators version 4.7.9 (Homepage ...
    CDO:                            Climate Data Operators version 1.7.2 (htt...
    history_of_appended_files:      Wed Jan 22 14:22:39 2020: Appended file t...
    id:                             doi:10.5065/D6SJ1JCH
    title:                          NA-CORDEX Raw NAM-22i RegCM4 HadGEM2-ES H...
    version:                        2.0

In [ ]:
## Create a small slice with the full time axis for local testing. 

#subset = ds.isel(lat=0, lon=0)
#subset
#subset.to_netcdf('/glade/u/home/bonnland/cal360_data.nc')

In [ ]:
def get_datemap_360_to_noleap():
    ''' Return an array of dates mapping days from the 360-Day calendar to the No-Leap calendar. '''

    # Choose any year with 365 days. 
    dummy_year = 1999

    # These are the days of the year that will be missing on the time axis for each year. 
    # The goal is to spread missing dates out evenly over each year.
    #
    # Modify specific dates as desired. 
    missing_dates = [date(dummy_year, 1, 31),
                     date(dummy_year, 3, 31),
                     date(dummy_year, 5, 31),
                     date(dummy_year, 8, 31),
                     date(dummy_year, 10, 31),]
    
    day_one = date(dummy_year, 1, 1)
    missing_dates_indexes = [(day - day_one).days for day in missing_dates]    

    datemap_indexes = np.setdiff1d(np.arange(365), missing_dates_indexes)
    
    dates = pd.date_range(f'1/1/{dummy_year}', f'12/31/{dummy_year}')
    assert(len(dates) == 365)
    
    date_map = dates[datemap_indexes]
    assert(len(date_map) == 360)
    return date_map


def convert_to_noleap(cftime360, datemap):
    newdate = datemap[cftime360.dayofyr - 1]
    converted = cftime.DatetimeNoLeap(year=cftime360.year, month=newdate.month, day=newdate.day)
    return converted

In [ ]:
datemap = get_datemap_360_to_noleap()
datemap[0]

In [ ]:
ds['time'] = [convert_to_noleap(t, datemap) for t in ds.time.values]
ds

In [ ]:
ds['time'][:60]

In [ ]:
ds["time"] = ds.indexes["time"].to_datetimeindex()
ds['time'][:60]